In [56]:
import re
import requests
from bs4 import BeautifulSoup
import json

# https://codebeautify.org/htmlviewer/#
# https://github.com/sandino/Markdown-Cheatsheet#lines
# http://stackabuse.com/reading-and-writing-json-to-a-file-in-python/
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-the-tree
    # [int(s) for s in str.split() if s.isdigit()]

In [57]:
persons_names = page.body.find_all("div", title=True)

In [58]:
for person in persons_names:
    title = person["title"]
    data_persons[title] = {}
print(data_persons)

{'Бабаева Сабина Кямильевна': {}, 'Бабаш Александр Владимирович': {}, 'Бабаян Мариам Эдуардовна': {}, 'Бабаян Тамара Мануковна': {}, 'Бабенко Артем Валерьевич': {}, 'Бабенко Максим Александрович': {}, 'Бабкина Татьяна Сергеевна': {}, 'Бабкова Галина Олеговна': {}, 'Бабушкин Валерий Валерьевич': {}, 'Бабякина Ольга Алексеевна': {}, 'Багин Александр Михайлович': {}, 'Багирокова Ирина Гаруновна': {}, 'Багратиони Константин Амиранович': {}, 'Багринцева Наталья Валерьевна': {}, 'Бадалова Анастасия Сергеевна': {}, 'Бадмаева Занда Сергеевна': {}, 'Баев Василий Иванович': {}, 'Баева Зинаида Ивановна': {}, 'Баевский Олег Артемович': {}, 'Бажанов Леонид Александрович': {}, 'Баженов Григорий Александрович': {}, 'Баженова Таисия Алексеевна': {}, 'Баженова-Сорокина Александра Дмитриевна': {}, 'Базавлюк Владимир Николаевич': {}, 'Базавлюк Дарья Николаевна': {}, 'Базаров Тахир Юсупович': {}, 'Базарова Гули Тахировна': {}, 'Базиева Зарета Аликовна': {}, 'Базилева Ирина Игоревна': {}, 'Базыкин Георгий 

In [59]:
page.body.find_all("div", class_="post person")

[<div class="post person">
 <div class="post__content post__content_person">
 <div class="l-extra small">
 <span>+7(495) 772-9590 *26061</span>
 <br/>
 <a class="link" data-at='["s","b","a","b","a","e","v","a","-at-","h","s","e",".","r","u"]'></a>
 </div>
 <div class="main content small">
 <div class="content__inner content__inner_foot1">
 <a class="link link_dark large b" href="/org/persons/3674675">
 <div alt="Бабаева Сабина Кямильевна" class="g-pic person-avatar-small2" style="background-image: url(/org/persons/cimage/3674675);" title="Бабаева Сабина Кямильевна">
 </div>
 					Бабаева Сабина Кямильевна
 				</a>
 <p class="with-indent7">
 <span>
 								Начальник учебной части:
 								
 									<a class="link" href="https://icef.hse.ru/">Международный институт экономики и финансов</a>
 									/
 								
 									учебная часть магистратуры
 									
 								
 							</span>
 </p>
 </div>
 </div>
 </div>
 </div>, <div class="post person">
 <div class="post__content post__con

In [60]:
persons = page.body.find_all("div", class_="post person")

In [61]:
def get_languages(link):
    url = "https://www.hse.ru" + link
    p = requests.get(url)
    person_page = BeautifulSoup(p.text, 'html.parser')
    
    # Владение языками
    try: 
        lang_list = person_page.body.find("dl", class_="main-list large main-list-language-knowledge-level").find_all("dd")
        lang_list = [lang.text for lang in lang_list]
    except AttributeError:
        lang_list = 'none'
    
    return lang_list
    
#get_languages('/org/persons/25477')

In [62]:
def get_contacts(link):
    url = "https://www.hse.ru" + link
    p = requests.get(url)
    person_page = BeautifulSoup(p.text, 'html.parser')
    contacts = {
        'phone_list': [],
        #'mail_list': [],
        'address': []
    }
    
    contacts_box = person_page.body.find("dl", class_="main-list large")
    #print(contacts_box.prettify())
    try:
        contents = contacts_box.find_all("dd")
        for box in contents: 
            if "Телефон" in box.text:
                phone_list = box.text.split(":")[1:]
                contacts['phone_list'] = phone_list

            elif "Электронная почта" in box.text:
                #mail_list = ['mail']
                #contacts['mail_list'] = mail_list
                continue

            elif "Адрес" in box.text:
                address = box.contents[0].split(":")[-1].strip()
                contacts['address'] = address
    except AttributeError:
        pass        
        
    return contacts

# Проблемы: Как достать почту? Нестабильность в кол-ве номеров телефонов -- может, не брать их вовсе?
# Что делать с теми, у кого нет инфы
    
get_contacts('/org/persons/73662404')

{'address': 'Старая Басманная ул., д. 21/4, стр. 5, каб. Б-307',
 'phone_list': ['+7499138359322699']}

In [65]:
def get_science_info(link):
    url = "https://www.hse.ru" + link
    p = requests.get(url)
    person_page = BeautifulSoup(p.text, 'html.parser')
    ids = {}
    
    extra_info = person_page.find("dl", class_="main-list person-extra-indent")
    try:
        ids_info = extra_info.find_all('dd')
    except AttributeError:
        advisor_id = 'none'
        return ids, advisor_id
    
    for dd in ids_info:
        try: 
            id_name = dd.span.text
            id_number = dd.a.text
            ids[id_name] = id_number
        except AttributeError:
            continue
    
    try:
        advisor_tag = person_page.find("dl", class_="main-list person-extra-indent").next_sibling
        advisor_id = advisor_tag.a['href']
    except TypeError:
        advisor_id = 'none'
    
    return ids, advisor_id
    
get_science_info('/org/persons/196917306')

({}, 'none')

In [63]:
def get_intersts(link):
    url = "https://www.hse.ru" + link
    p = requests.get(url)
    person_page = BeautifulSoup(p.text, 'html.parser')
    interests = {}
    
    try:
        interests_div = person_page.find("div", class_="b-person-data__inner b-person-data__tags")
        interests_tags = interests_div.find_all('a')
        for a in interests_tags:
            interest_name = a.text
            interest_id = a['href'].split("=")[1]
            interests[interest_name] = interest_id
    except AttributeError:
        pass
        
    return interests
    
get_intersts('/staff/aparinov')

{'алгоритмы': '71848768',
 'анализ данных': '61913648',
 'анализ формальных понятий': '61296475'}

In [ ]:
# Лысая /org/persons/196917306
# Тетя  /org/persons/25477
# Паринов /staff/aparinov

In [64]:
with open("persons_Б.txt", 'w', encoding="utf-8") as outfile:
    for person in persons:
        name = person.find("div", title=True)["title"]
        link = person.find("a", href=True)["href"]

        lang_list = get_languages(link)
        contacts_dict = get_contacts(link)
        science_dict, advisor = get_science_info(link)
        interests_dict = get_intersts(link)


        data_persons[name] = {
            "link": link,
            "languages": lang_list,
            "contacts": contacts_dict,
            "science_ids": science_dict,
            "advisor": advisor,
            "interests": interests_dict


        }
        print('DONE', name)
        
    json.dump(data_persons, outfile, ensure_ascii=False)    
print("ALL DONE")

    
#print(data_persons)

DONE Бабаева Сабина Кямильевна
DONE Бабаш Александр Владимирович
DONE Бабаян Мариам Эдуардовна
DONE Бабаян Тамара Мануковна
DONE Бабенко Артем Валерьевич
DONE Бабенко Максим Александрович
DONE Бабкина Татьяна Сергеевна
DONE Бабкова Галина Олеговна
DONE Бабушкин Валерий Валерьевич
DONE Бабякина Ольга Алексеевна
DONE Багин Александр Михайлович
DONE Багирокова Ирина Гаруновна
DONE Багратиони Константин Амиранович
DONE Багринцева Наталья Валерьевна
DONE Бадалова Анастасия Сергеевна
DONE Бадмаева Занда Сергеевна
DONE Баев Василий Иванович
DONE Баева Зинаида Ивановна
DONE Баевский Олег Артемович
DONE Бажанов Леонид Александрович
DONE Баженов Григорий Александрович
DONE Баженова Таисия Алексеевна
DONE Баженова-Сорокина Александра Дмитриевна
DONE Базавлюк Владимир Николаевич
DONE Базавлюк Дарья Николаевна
DONE Базаров Тахир Юсупович
DONE Базарова Гули Тахировна
DONE Базиева Зарета Аликовна
DONE Базилева Ирина Игоревна
DONE Базыкин Георгий Александрович
DONE Байбикова Татьяна Николаевна
DONE Ба

DONE Благодетелева Елизавета Дмитриевна
DONE Благодир Алла Леонтьевна
DONE Бланк Михаил Львович
DONE Блескина Ирина Александровна
DONE Блинкин Михаил Яковлевич
DONE Блинникова Елена Егоровна
DONE Блинова Анна Михайловна
DONE Блок Эвелина Евгеньевна
DONE Блохин Андрей Алексеевич
DONE Блохина Екатерина Тимофеевна
DONE Блюхер Юлия Николаевна
DONE Бобер Станислав Алексеевич
DONE Бобрик-Фремке Марина Анатольевна
DONE Бобров Владислав Вячеславович
DONE Боброва Екатерина Юрьевна
DONE Бобровник Михаил Валериевич
DONE Бобрянок Татьяна Анатольевна
DONE Бобылева Ксения Владимировна
DONE Богатырев Вадим Петрович
DONE Богатырева Инна Яковлевна
DONE Богачев Владимир Игоревич
DONE Богачева Нина Владимировна
DONE Богачёв Максим Игоревич
DONE Богданов Алексей Валентинович
DONE Богданов Анвяр Равильевич
DONE Богданова Татьяна Кирилловна
DONE Богдановская Ирина Юрьевна
DONE Богоевич Марина Алексеевна
DONE Боголепова Светлана Викторовна
DONE Боголюбов Дмитрий Петрович
DONE Богомазова Вероника Львовна
DONE

In [ ]:
with open("persons_A.txt", 'w', encoding="utf-8") as outfile:
    json.dump(data_persons, outfile, ensure_ascii=False)

In [66]:
letters_list = ["В", "Г", "Д", "Е", "Ж", "З", "И", "К", "Л", "М", "Н", "О", "П", "Р", "С", "Т",
                "У", "Ф", "Х", "Ц", "Ч", "Ш", "Щ", "Э", "Ю", "Я"]

In [ ]:
for letter in letters_list:
    print("START", letter)
    link = "https://www.hse.ru/org/persons/?ltr=" + letter + ";udept=22726"
    data_persons = {}
    
    r = requests.get(link)
    page = BeautifulSoup(r.text, 'html.parser')
    persons_names = page.body.find_all("div", title=True)
    for person in persons_names:
        title = person["title"]
        data_persons[title] = {}
    
    persons = page.body.find_all("div", class_="post person")
    
    for person in persons:
        name = person.find("div", title=True)["title"]
        link = person.find("a", href=True)["href"]

        lang_list = get_languages(link)
        contacts_dict = get_contacts(link)
        science_dict, advisor = get_science_info(link)
        interests_dict = get_intersts(link)


        data_persons[name] = {
            "link": link,
            "languages": lang_list,
            "contacts": contacts_dict,
            "science_ids": science_dict,
            "advisor": advisor,
            "interests": interests_dict
        }
        
    with open("persons_"+letter+".txt", 'w', encoding="utf-8") as outfile:
        json.dump(data_persons, outfile, ensure_ascii=False)
        
    print('DONE', letter)

START В
